<a href="https://colab.research.google.com/github/sh3zaad/pypackage/blob/master/Classification_Hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Xente Hackathon

## Honour Code

### Honour Code

I SHEZAAD, ESSOP MOOSA, confirm - by submitting my - that the solutions in this notebook are a result of my own work and that I abide by the EDSA honour code (https://drive.google.com/file/d/1QDCjGZJ8-FmJE3bZdIQNwnJyQKPhHZBn/view?usp=sharing).

Non-compliance with the honour code constitutes a material breach of contract.

## Importing Libraries

In [2]:
import numpy as np
import pandas as pd


import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier


from sklearn.model_selection import train_test_split


from sklearn.utils import resample
from imblearn.combine import SMOTETomek


from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report


import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


## Loading Data into Colab

In [0]:
# from google.colab import files
# uploaded = files.upload()

## Creating dataframes for data

In [0]:
import io

train_df = pd.read_csv('training.csv')
test_df = pd.read_csv('test.csv')

In [5]:
train_df.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,UGX,256,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,1000,2018-11-15T02:18:49Z,2,0
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,20,2018-11-15T02:19:08Z,2,0
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,UGX,256,ProviderId_6,ProductId_1,airtime,ChannelId_3,500.0,500,2018-11-15T02:44:21Z,2,0
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,UGX,256,ProviderId_1,ProductId_21,utility_bill,ChannelId_3,20000.0,21800,2018-11-15T03:32:55Z,2,0
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-644.0,644,2018-11-15T03:34:21Z,2,0


In [6]:
train_df.shape

(95662, 16)

In [7]:
test_df.shape

(45019, 15)

Combining train and test to perform pre-processing and engineering.

In [0]:
X = pd.concat([train_df.iloc[:,0:-1], test_df], axis = 0)
y = train_df['FraudResult']

In [9]:
X.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,UGX,256,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,1000,2018-11-15T02:18:49Z,2
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,20,2018-11-15T02:19:08Z,2
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,UGX,256,ProviderId_6,ProductId_1,airtime,ChannelId_3,500.0,500,2018-11-15T02:44:21Z,2
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,UGX,256,ProviderId_1,ProductId_21,utility_bill,ChannelId_3,20000.0,21800,2018-11-15T03:32:55Z,2
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-644.0,644,2018-11-15T03:34:21Z,2


In [10]:
X.shape

(140681, 15)

## Missing Values

In [11]:
X.isnull().any()

TransactionId           False
BatchId                 False
AccountId               False
SubscriptionId          False
CustomerId              False
CurrencyCode            False
CountryCode             False
ProviderId              False
ProductId               False
ProductCategory         False
ChannelId               False
Amount                  False
Value                   False
TransactionStartTime    False
PricingStrategy         False
dtype: bool

No missing values.

## Pre-Processing, EDA and Feature-Engineering

From Variable definitions see that Transaction is unique identifier.
Checking to see if any other column is also unique as Variable definition document is not very clear.

In [12]:
for col in X.columns:
  print(col, X[col].nunique() == len(X['TransactionId']))

TransactionId True
BatchId False
AccountId False
SubscriptionId False
CustomerId False
CurrencyCode False
CountryCode False
ProviderId False
ProductId False
ProductCategory False
ChannelId False
Amount False
Value False
TransactionStartTime False
PricingStrategy False


Looking at data types.

See that majority of columns are categorical (object here but on closer inspection they are categorical).

In [13]:
X.dtypes

TransactionId            object
BatchId                  object
AccountId                object
SubscriptionId           object
CustomerId               object
CurrencyCode             object
CountryCode               int64
ProviderId               object
ProductId                object
ProductCategory          object
ChannelId                object
Amount                  float64
Value                     int64
TransactionStartTime     object
PricingStrategy           int64
dtype: object

In [14]:
X.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,UGX,256,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,1000,2018-11-15T02:18:49Z,2
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,20,2018-11-15T02:19:08Z,2
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,UGX,256,ProviderId_6,ProductId_1,airtime,ChannelId_3,500.0,500,2018-11-15T02:44:21Z,2
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,UGX,256,ProviderId_1,ProductId_21,utility_bill,ChannelId_3,20000.0,21800,2018-11-15T03:32:55Z,2
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-644.0,644,2018-11-15T03:34:21Z,2


Notice that 'CurrencyCode' and 'CountryCode' are representing the same information. Not only that but also the information is the same for every observation. So these columns can be dropped.


In [15]:
X['CurrencyCode'].nunique()

1

In [0]:
X.drop(['CurrencyCode', 'CountryCode'], axis = 1, inplace = True)

Notice that there is a time column. *Time* to convert this to usable columns.

In [0]:
X['TransactionStartTime'] = pd.to_datetime(X['TransactionStartTime'])

Will be splitting the datetime column into Hour, Day, Month, Year and dropping the datetime column.

In [0]:
X['Year'] = X['TransactionStartTime'].apply(lambda x: x.year)
X['Month'] = X['TransactionStartTime'].apply(lambda x: x.month)
X['Day'] = X['TransactionStartTime'].apply(lambda x: x.day)
X['Hour'] = X['TransactionStartTime'].apply(lambda x: x.hour)
X['Dayofweek'] = X['TransactionStartTime'].apply(lambda x: x.dayofweek)

In [0]:
X.drop('TransactionStartTime', axis = 1, inplace = True)

From the Variables definitions, see that the column Value is supposed to just be the absolute value of the Amount column. But it was noticed that there is a discrepancy with the value in Values and the absolute value of the Amount column. This could be due to bank charge or other hidden costs.
With this in mind in order not to let these hidden charges affect the model, just the Amount column will be used.

In [0]:
X.drop('Value', axis = 1, inplace = True)

In [21]:
X.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,ProviderId,ProductId,ProductCategory,ChannelId,Amount,PricingStrategy,Year,Month,Day,Hour,Dayofweek
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,2,2018,11,15,2,3
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,2,2018,11,15,2,3
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,ProviderId_6,ProductId_1,airtime,ChannelId_3,500.0,2,2018,11,15,2,3
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,ProviderId_1,ProductId_21,utility_bill,ChannelId_3,20000.0,2,2018,11,15,3,3
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-644.0,2,2018,11,15,3,3


After looking at the Variable Definitions, SubscriptionId and AccountId could possibly be linked. Taking a closer look at these two columns.

In [22]:
X['AccountId'].nunique()

4841

In [23]:
X['SubscriptionId'].nunique()

4836

The two columns do not completely represent the same information but a correlation between the two could to be done.

Using Cramer's V Statistic (determines the strength of association after chi-square has determined significance):

In [24]:
import scipy.stats as ss

def cramers_v(confusion_matrix):
    """ calculate Cramers V statistic for categorial-categorial association.
        uses correction from Bergsma and Wicher,
        Journal of the Korean Statistical Society 42 (2013): 323-328
    """
    chi2 = ss.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))
    rcorr = r - ((r-1)**2)/(n-1)
    kcorr = k - ((k-1)**2)/(n-1)
    return np.sqrt(phi2corr / min((kcorr-1), (rcorr-1)))

confusion_matrix = pd.crosstab(X['AccountId'], X['SubscriptionId']).as_matrix()
print(cramers_v(confusion_matrix))

# Code sourced directly from https://stackoverflow.com/questions/46498455/categorical-features-correlation

0.9999039422894663


There is a strong correlation between the two columns thus one of these columns will be dropped.
Keeping AccountId as it pertains to information about the account itself and not on a package/subscription (more pertinent information).

In [0]:
X.drop('SubscriptionId', axis = 1, inplace = True)

In [26]:
X.head()

,TransactionId,BatchId,AccountId,CustomerId,ProviderId,ProductId,ProductCategory,ChannelId,Amount,PricingStrategy,Year,Month,Day,Hour,Dayofweek
0,TransactionId_76871,BatchId_36123,AccountId_3957,CustomerId_4406,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,2,2018,11,15,2,3
1,TransactionId_73770,BatchId_15642,AccountId_4841,CustomerId_4406,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,2,2018,11,15,2,3
2,TransactionId_26203,BatchId_53941,AccountId_4229,CustomerId_4683,ProviderId_6,ProductId_1,airtime,ChannelId_3,500.0,2,2018,11,15,2,3
3,TransactionId_380,BatchId_102363,AccountId_648,CustomerId_988,ProviderId_1,ProductId_21,utility_bill,ChannelId_3,20000.0,2,2018,11,15,3,3
4,TransactionId_28195,BatchId_38780,AccountId_4841,CustomerId_988,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-644.0,2,2018,11,15,3,3


Lastly looking at Variable definitions, see that BathId is simply just the id for how the transactions were processed, so this column is dropped.

In [0]:
X.drop('BatchId', axis = 1, inplace = True)

Now to encode the categorical data.

In [0]:
X_dummies = pd.get_dummies(X[['AccountId', 'CustomerId', 'ProviderId', 'ProductId', 'ProductCategory', 'ChannelId', 'PricingStrategy', 'Year', 'Month', 'Day', 'Hour', "Dayofweek"]], drop_first = True)

In [0]:
X = pd.concat([X[['TransactionId', 'Amount']], X_dummies], axis = 1)

In [30]:
X.head()

,TransactionId,Amount,PricingStrategy,Year,Month,Day,Hour,Dayofweek,AccountId_AccountId_10,AccountId_AccountId_100,AccountId_AccountId_1000,AccountId_AccountId_1001,AccountId_AccountId_1002,AccountId_AccountId_1003,AccountId_AccountId_1004,AccountId_AccountId_1005,AccountId_AccountId_1006,AccountId_AccountId_1007,AccountId_AccountId_1008,AccountId_AccountId_1009,AccountId_AccountId_101,AccountId_AccountId_1010,AccountId_AccountId_1011,AccountId_AccountId_1012,AccountId_AccountId_1013,AccountId_AccountId_1014,AccountId_AccountId_1015,AccountId_AccountId_1016,AccountId_AccountId_1017,AccountId_AccountId_1018,AccountId_AccountId_1019,AccountId_AccountId_102,AccountId_AccountId_1020,AccountId_AccountId_1021,AccountId_AccountId_1022,AccountId_AccountId_1023,AccountId_AccountId_1024,AccountId_AccountId_1025,AccountId_AccountId_1026,AccountId_AccountId_1027,...,ProviderId_ProviderId_6,ProductId_ProductId_10,ProductId_ProductId_11,ProductId_ProductId_12,ProductId_ProductId_13,ProductId_ProductId_14,ProductId_ProductId_15,ProductId_ProductId_16,ProductId_ProductId_17,ProductId_ProductId_18,ProductId_ProductId_19,ProductId_ProductId_2,ProductId_ProductId_20,ProductId_ProductId_21,ProductId_ProductId_22,ProductId_ProductId_23,ProductId_ProductId_24,ProductId_ProductId_25,ProductId_ProductId_26,ProductId_ProductId_27,ProductId_ProductId_3,ProductId_ProductId_4,ProductId_ProductId_5,ProductId_ProductId_6,ProductId_ProductId_7,ProductId_ProductId_8,ProductId_ProductId_9,ProductCategory_data_bundles,ProductCategory_financial_services,ProductCategory_movies,ProductCategory_other,ProductCategory_retail,ProductCategory_ticket,ProductCategory_transport,ProductCategory_tv,ProductCategory_utility_bill,ChannelId_ChannelId_2,ChannelId_ChannelId_3,ChannelId_ChannelId_4,ChannelId_ChannelId_5
0,TransactionId_76871,1000.0,2,2018,11,15,2,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,TransactionId_73770,-20.0,2,2018,11,15,2,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0
2,TransactionId_26203,500.0,2,2018,11,15,2,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,TransactionId_380,20000.0,2,2018,11,15,3,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
4,TransactionId_28195,-644.0,2,2018,11,15,3,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0


## Splitting the Data

In [0]:
train = X[:train_df.shape[0]]
test = X[train_df.shape[0]:]

Create a train-test split.

In [0]:
X_train, X_test, y_train, y_test = train_test_split(train, y, test_size=0.20, shuffle=False)

## Balancing labels

**The split is done before balancing so not to add oversamples into the validation set.**

Checking the balance of the labels see that the data is heavily unbalanced.

In [33]:
y_train.value_counts()

0    76402
1      127
Name: FraudResult, dtype: int64

Using SMOTE with undersampling to balance the data.
SMOTE oversamples with synthetic data. This is done by creating synthetic sample based off the original samples. But before this is done the majrity class is undersampled to a manageable number.

First we undersample as their is a lot of data to get through and since working with limited memory it would be best to undersample the extremely lare majority class before implementing smote.

In [0]:
training = pd.concat([X_train, y_train.to_frame(name = 'FraudResult')], axis = 1)

In [0]:
# Dropping TransactionId as it is just an index but don't want to set as index as it takes up too much memory.
not_fraud = training[training['FraudResult']==0].drop('TransactionId', axis = 1)
fraud = training[training['FraudResult']==1].drop('TransactionId', axis = 1)

X_test_1 = X_test.drop('TransactionId', axis = 1)

In [0]:
not_fraud_downsampled = resample(not_fraud,
                                replace = False, # sample without replacement
                                n_samples = 1000, random_state = 42)

In [0]:
downsampled = pd.concat([not_fraud_downsampled, fraud])

In [0]:
X_train_down = downsampled.drop('FraudResult', axis = 1)

In [0]:
y_train_down = downsampled['FraudResult']

Implementing SMOTE to oversample the minority class.

In [0]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=42, ratio=1.0)
X_train_sm, y_train_sm = sm.fit_sample(X_train_down, y_train_down)

In [41]:
X_train_sm.shape

(2000, 12369)

## Modelling

Applying three models: RandomForest, Logistic Regression and KNN.

Using f1-score as the metric of performance.

Also performing some hyperparameter tuning for the best performing model.

### Random Forest

In [0]:
rf = RandomForestClassifier(random_state=42)

In [43]:
rf.fit(X_train_down, y_train_down)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [0]:
X_test_1 = X_test.drop('TransactionId', axis = 1)

In [45]:
print(f1_score(y_train_down, rf.predict(X_train_down)))
print(f1_score(y_test, rf.predict(X_test_1)))

0.9920634920634921
0.4025974025974026



### Logistic Regression

In [0]:
lm = LogisticRegression(random_state = 42)

In [47]:
lm.fit(X_train_down, y_train_down)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=42, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [48]:
print(f1_score(y_train_down, lm.predict(X_train_down)))
print(f1_score(y_test, lm.predict(X_test_1)))

0.8927038626609441
0.8627450980392156


### KNN

In [0]:
knn = KNeighborsClassifier()

In [50]:
knn.fit(X_train_down, y_train_down)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [0]:
print(f1_score(y_train_down, knn.predict(X_train_down)))
print(f1_score(y_test, knn.predict(X_test_1)))

0.9606299212598425


From the above, it can be seen that logistic regression performs the best without any issues of overfitting and a relatively high score.

### Hyper-parameter Tuning

In [0]:
param_grid_lm = {'penalty' : ['l1', 'l2'], 'C' : [0.00001, 0.00005, 0.00008, 0.0001, 0.001,0.005,0.01,0.1,1,10,100,1000]}

In [0]:
clf_lm = GridSearchCV(lm, param_grid = param_grid_lm, cv = 5, verbose=True, n_jobs=-1)

In [0]:
clf_lm.fit(X_train_down, y_train_down)

In [0]:
print(f1_score(y_train_down, clf_lm.predict(X_train_down)))
print(f1_score(y_test, clf_lm.predict(X_test_1)))

A decrease in performance. This could indicate that the default parameters for logistic regression are already close to the optimised parameters.

## Submission

Based off the performance of the models above, logistic regression will be used with default parameter values.

In [0]:
submission = lm.predict(test.drop('TransactionId', axis = 1))

In [0]:
sub_df = pd.DataFrame(submission, columns = ['FraudResult'])

In [0]:
submission_df = pd.concat([test['TransactionId'].to_frame(), sub_df], axis = 1)

In [0]:
submission_df.head()

In [0]:
submission_csv = submission_df.to_csv('submission_logreg.csv', index = False)

In [0]:
from google.colab import files
files.download("submission_logreg.csv")

## Conclusion

A model was trained to predict farud cases based off data obtained via an app which records time of transaction, account type, customer id's, etc.

The best performing model was logistic regression and after hyperparameter tuning it was found that the default parameter values were the already optimised. Perhaps doing hyperparameter tuning closer to these default values could result in better performance but due to time constraints this was not done here.

The data labels were extremely unbalanced, with the minority class only making a small chunk of the full data thus oversampling with undersampling had to be done.